In [2]:
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
import scipy.sparse as sp
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
def random_subset(adata,cell_num=1200):
    random_indices = np.random.choice(adata.n_obs, cell_num, replace=True)   # 从 276593 个细胞中随机选取 3000 个细胞的索引
    adata_subset = adata[random_indices, :]   # 使用选取的索引创建新的 AnnData 对象

    return adata_subset

In [5]:
dataset_dir = '/home/hanyuji/Data/ST_data/marmoset_hippocampus/cell/marmoset1/'

cell_macaque1 = ['T447.h5ad', 'T452.h5ad', 'T456.h5ad', 'T460.h5ad', 'T463.h5ad', 'T466.h5ad', 'T472.h5ad', 'T475.h5ad', 'T479.h5ad', 'T448.h5ad',
 'T454.h5ad', 'T457.h5ad', 'T461.h5ad', 'T464.h5ad', 'T467.h5ad', 'T473.h5ad', 'T476.h5ad', 'T480.h5ad', 'T451.h5ad', 'T455.h5ad', 'T458.h5ad',
 'T462.h5ad', 'T465.h5ad', 'T470.h5ad', 'T474.h5ad', 'T478.h5ad', 'T482.h5ad']

adatas = []
for file_name in tqdm(cell_macaque1):
    adata = sc.read_h5ad(dataset_dir+file_name)
    sc.pp.filter_genes(adata, min_counts = 5)
    subset = random_subset(adata,1000)
    adatas.append(subset)

combined_adata = ad.concat(
    adatas, 
    axis=0, 
    join='outer', 
    merge='unique', 
    uns_merge='unique', 
    label='batch', 
    keys=list(range(1, len(adatas)+1)), 
    index_unique='-', 
    fill_value=0
)


100%|██████████| 27/27 [00:15<00:00,  1.75it/s]
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
combined_adata

AnnData object with n_obs × n_vars = 27000 × 24272
    obs: 'region', 'cellID', 'x', 'y', 'cellType', 'batch'
    obsm: 'spatial'

In [7]:
sc.pp.highly_variable_genes(combined_adata, n_top_genes=2000 ,flavor='seurat_v3')
combined_adata = combined_adata[:, combined_adata.var.highly_variable]
# 标准化 log 
sc.pp.normalize_total(combined_adata, target_sum=1e4)
sc.pp.log1p(combined_adata)

/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:233: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


In [8]:
combined_adata.write('/home/hanyuji/Workbench/ST/paste_alignment_cortex/PASTE_align/marmoset_hippocampus_cell1_27slices_1000spot_2000gene.h5ad')